dependencies </n>
Env :: Forecastinit
Python=3.11.5

Libraries

In [208]:
# General 
import pandas as pd
import numpy as np

# sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# skforecast 
from skforecast.datasets import load_demo_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

# ML Models 
from xgboost import XGBRegressor

Input

In [209]:
Historical_data_org = pd.read_csv('Input/Actual.csv')
TimeKey_org = pd.read_csv('Input/Time.csv')
Forecast_level = ['Version.[Version Name]', 'Channel.[Channel]', 'Account.[Account]',
       'PnL.[PnL]', 'Demand Domain.[Demand Domain]', 'Region.[Region]',
       'Location.[Location]', 'Time.[Planning Month]', 'Item.[Item]']
Time_column = ['Time.[Planning Month]']
Historical_data_column = ['Actual']
#Date should be in format of time key column, which is available in TimeKey file 
Time_key_column_name = "Time.[PlanningMonthKey]"
Time_key_date_format = "%m/%d/%Y %I:%M:%S %p"
Historic_start_date = "M09-20" 
Historic_end_date = "M12-22"
Forecast_start_date = "M01-23"
Forecast_end_date = "M03-23"
drivers = ['driver_Month']
models = ["Random Forest", "XG Boost"]


Custom Functions

In [222]:
# Data filter based on start and end date, then split in X and y based on provided columns 
# split(historical data frame contains drivers and actual, start date , end date, X columns in list, y column in string )
def split(data,start_date_key,end_date_key,X_cols,y_col):
    data = data[(data['TimeKey']>=start_date_key)&(data['TimeKey']<=end_date_key)]
    X = data[['key','TimeKey']+X_cols]
    y = data[[y_col]]
    return X, y

In [228]:
# SK Forecast ML Models
# skforecastpredict(models in list, X train data frame, y train data frame, X test data frame ) 
def skforecastpredict(models, X_train, y_train, X_test):
    # Predicted output data frame 
    Output = pd.DataFrame()

    # converting data frame to series, as sk forecast y accept series 
    y_train_loc = y_train[y_train.columns.values[0]]
    
    # from sklearn.preprocessing import FunctionTransformer
    # transformer_y = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

    # Random Forest ========================================================================================================================= 
    if "Random Forest" in models:
        randomforestforecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = [2]
             )
        randomforestforecaster.fit(y= y_train_loc)
        y_hat_randomforest = randomforestforecaster.predict_interval(steps=11).reset_index(drop=True)
        y_hat_randomforest.columns = ['Random Forest Y_hat', 'Random Forest Lower Bound', 'Random Forest Upper Bound']
        Output = pd.concat([Output, y_hat_randomforest], axis=1)
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    
    #XG Boost ================================================================================================================================
    if "XG Boost" in models:
        xgboostForecaster = ForecasterAutoreg(
                regressor= XGBRegressor(random_state = 213),
                lags= [1,2]
            )      
        xgboostForecaster.fit(y=y_train_loc)
        y_hat_xgboost = xgboostForecaster.predict_interval(steps=11).reset_index(drop=True)
        y_hat_xgboost.columns = ['XG Boost Y_hat', 'XG Boost Lower Bound', 'XG Boost Upper Bound']
        Output = pd.concat([Output, y_hat_xgboost], axis=1)
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
        
    return Output

Processing Input

In [212]:
Forecast_level_minus_time = Forecast_level.copy()
Forecast_level_minus_time.remove(Time_column[0])
Forecast_level_minus_time

['Version.[Version Name]',
 'Channel.[Channel]',
 'Account.[Account]',
 'PnL.[PnL]',
 'Demand Domain.[Demand Domain]',
 'Region.[Region]',
 'Location.[Location]',
 'Item.[Item]']

Copy Input 

In [213]:
Historical_data = Historical_data_org.copy(deep=True)
TimeKey = TimeKey_org.copy(deep=True)
TimeKey['TimeKey'] = pd.to_datetime(TimeKey[Time_key_column_name], format=Time_key_date_format)

In [214]:
# calculating Historic_end_date_key
Historic_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_start_date]['TimeKey'].values[0])
Historic_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_end_date]['TimeKey'].values[0])
Forecast_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_start_date]['TimeKey'].values[0])
Forecast_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_end_date]['TimeKey'].values[0])

Processing Copy Input

In [215]:
Historical_data[Historical_data_column] = Historical_data[Historical_data_column].astype(float)


Creating Key in Historical Data

In [216]:
# creating keys 
Historical_data['key'] = Historical_data[Forecast_level_minus_time].astype(str).agg("__ MDJoinner__".join, axis=1)
# dropping columns, which is already present in keys 
Historical_data.drop(Forecast_level_minus_time,axis=1,inplace=True) 
Historical_data.head(2)

,Time.[Planning Month],Actual,key
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...


Custom Filter

In [217]:
Historical_data = Historical_data[Historical_data['key']=='CurrentWorkingView__ MDJoinner__B2B__ MDJoinner__AMO__ MDJoinner__DP__ MDJoinner__DP__ MDJoinner__ShipTo1__ MDJoinner__DP__ MDJoinner__Loctite 248 19g Stick']
# Historical_data['key'].values

Merging Time Key in Historical Data

In [218]:
Historical_data = pd.merge(Historical_data,TimeKey[Time_column+["TimeKey"]],on=Time_column,how='left')
Historical_data.head(2)

,Time.[Planning Month],Actual,key,TimeKey
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02


In [219]:
# Creating drivers 
Historical_data['driver_Month'] = Historical_data['TimeKey'].dt.month
Historical_data.head()

,Time.[Planning Month],Actual,key,TimeKey,driver_Month
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05,7
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02,8
2,M09-20,462.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-30,8
3,M10-20,174.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-10-04,10
4,M11-20,179.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-11-01,11


Train and Test Data

In [220]:
X_train, y_train = split(Historical_data, Historic_start_date_key,Historic_end_date_key,drivers,Historical_data_column[0])
X_test,y_test = split(Historical_data, Forecast_start_date_key, Forecast_end_date_key, drivers, Historical_data_column[0])

Prediction

In [229]:
skforecastpredict(models,X_train, y_train, X_test)

,Random Forest Y_hat,Random Forest Lower Bound,Random Forest Upper Bound,XG Boost Y_hat,XG Boost Lower Bound,XG Boost Upper Bound
0,242.180,171.91925,416.80000,241.001587,240.999939,241.003296
1,275.090,206.50500,449.71000,228.000656,227.999008,228.002365
2,261.680,120.64050,566.39950,245.999619,245.997971,246.001511
3,539.310,146.78275,667.02900,209.000092,208.998444,209.001801
4,244.500,153.08075,621.96000,167.000000,166.998352,167.001709
5,452.380,131.19425,629.60000,174.999649,174.997864,175.001541
6,225.340,138.79750,609.27000,231.998749,231.997101,232.000458
7,221.270,137.12175,571.63900,539.998108,539.996460,539.999817
8,215.460,141.00500,595.91575,731.998291,731.996643,732.000000
9,202.755,139.73150,627.32000,626.998901,626.997116,627.000610
